#### This code solves the nonlinear Schrödinger equation with natural boundary conditions: $u_{x}(x_L,t) = u_{x}(x_R,t) = 0$ for all $t$, using a FEM semi-discretization  in space and different time integrators in time:
\begin{align}\label{Schrodinger_eqn}
    i u_t + u_{xx} + \beta |u|^2 u & = 0, \text{on} \ [x_L,x_R] \times (0,T], \\
    u(x,0) = \textrm{sech}(x).
\end{align}

#### This equation has multi-soliton solutions for different values of $\beta$. This code generates data used to produce Figures 7, 8, and 9, as well as Table 5 in the manuscript.


#### Required libraries

In [1]:
# Required libraries 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from IPython.display import clear_output, display, Math
from TimeIntegrators import ImEx_schemes, Op_Sp_Coeff

#### Choose the case: 2-, or  3- solitons 

In [2]:
# Choose a particular case
#q = 8; sol = 2; inv = 2
q = 18; sol = 3; inv = 2

# time-stepping methods
Mthds_B = ['ImEx3','ImEx4']; Mthds_MR = ['ImEx3(MR)','ImEx4(MR)'];
Mthds_MR_EC = ['ImEx3(MR)(EC)','ImEx4(MR)(EC)']; Mthds_SP = ['S2','AK4']
Stage = [4,6]; Order = [3,4]; em_Or = [2,3]; Sch_No = [3,4]
SP_Stage = [2,5]; SP_Order = [2,4]; SP_Sch_No = [1,2];  

# Different casese
if q == 8 and sol == 2 and inv == 2: 
    xL = -35; xR = 35; L = xR-xL; m1 = 16; N =m1*L; t0 = 0; DT = [0.01,0.05]; SP_DT = [0.05,0.05]; T = 5
elif q == 18 and sol == 3 and inv == 2: 
    xL = -35; xR = 35; L = xR-xL; m1 = 32; N =m1*L; t0 = 0; DT = [0.01,0.01]; SP_DT = [0.01,0.01] ; T = 5

#### FEM semi-discretized matrices

In [3]:
dx = 1./m1; m = int((xR-xL)/dx); x = np.linspace(xL,xR,m); dx = x[1] - x[0]

# The matrix I_tilde
It = np.eye(2*m)
It[0][0] = 0.5; It[1][1] = 0.5
It[-1][-1] = 0.5; It[-2][-2] = 0.5
Itinv = np.diag(1./np.diag(It))

# The matrix S
A = np.array([[0, 1.], [-1., 0]])
S = np.zeros((2*m,2*m))
for j in range(1,m-1):
    S[2*j:2*j+2,2*j:2*j+2] = -2*A[:,:]
    S[2*j:2*j+2,2*j+2:2*j+4] = A[:,:]
    S[2*j:2*j+2,2*j-2:2*j] = A[:,:]
    
S[:2,:2] = -A[:,:]  
S[-2:,-2:] = -A[:,:]

S[:2,2:4] = A
S[-2:,-4:-2] = A
ItinvS = np.dot(Itinv,S) # I_tilde^(-1)*S

I = np.eye(2*m)
M = -1./dx**2 * ItinvS

#### Required functions

In [4]:
# The stiff part of the right hand side of the system of ODEs: du/dt = rhs
def rhs_stiff(U,dx):  
    ret = -1./dx**2 * np.dot(ItinvS,U) 
    return ret

 # The non-stiff part of the right hand side of the system of ODEs: du/dt = rhs
I1 = np.eye(m)
I2 = np.array([[1,1.],[1,1]])
A1 = np.kron(I1,A)
A2 = np.kron(I1,I2)

def rhs_non_stiff(U,q): 
    ret = -q*np.multiply(np.dot(A1,U),np.dot(A2,np.square(U)))
    return ret

def eta1(U,dx):  # Invariant: first quantity
    V = U[::2]
    W = U[1::2]
    return dx*np.sum(V**2+W**2)

def eta2(U,dx,q):   # Invariant: second quantity
    V = U[::2]
    W = U[1::2]
    return np.sum((np.diff(V)**2+np.diff(W)**2)/dx - 0.5*q*dx*(V[:-1]**2+W[:-1]**2)**2)

def rgam(gamma,u,inc,E_old,inv,q):
    uprop = u + np.dot(gamma,inc)  # u propsed
    uprop.shape
    if inv == 1:
        E_prop = np.array([eta1(uprop,dx)])
    elif inv == 2:
        E_prop = np.array([eta1(uprop,dx),eta2(uprop,dx,q)])
    return E_prop-E_old

def NLS_True_Sol(t,x,q):
    if q == 1:
        a = 1; c = 3; x0 = -10; theta0 = np.pi/4 
        ut = np.zeros(len(x))
        for i, xp in enumerate(x):
            ut[i] = np.sqrt(2)*np.exp(1j*((3/2)*(xp-c*t-x0)+theta0))*np.exp(1j*(a+c**2/4)*t) \
            /np.cosh(xp-c*t-x0)  
    elif q == 2:
        ut = np.exp(1j*t)/np.cosh(x)
    elif q == 8:
        sechx = 1./np.cosh(x)
        ut = np.exp(1j*t)*sechx*( 1+(3/4)*sechx**2*(np.exp(8*1j*t)-1) )/( 1-(3/4)*sechx**4*np.sin(4*t)**2 )
    elif q == 18:
        ut = (2*(3*np.exp(t*25*1j)*np.exp(x) + 15*np.exp(t*9*1j)*np.exp(9*x) + 48*np.exp(t*25*1j)*np.exp(7*x) + 48*np.exp(t*25*1j)*np.exp(11*x) + 24*np.exp(t*33*1j)*np.exp(3*x) + 54*np.exp(t*33*1j)*np.exp(5*x) + 3*np.exp(t*25*1j)*np.exp(17*x) + 54*np.exp(t*33*1j)*np.exp(13*x) + 24*np.exp(t*33*1j)*np.exp(15*x) + 135*np.exp(t*41*1j)*np.exp(9*x) + 30*np.exp(t*49*1j)*np.exp(5*x) + 120*np.exp(t*49*1j)*np.exp(7*x) + 120*np.exp(t*49*1j)*np.exp(11*x) + 30*np.exp(t*49*1j)*np.exp(13*x) + 60*np.exp(t*57*1j)*np.exp(9*x)))/(3*(np.exp(t*24*1j) + 10*np.exp(6*x) + 10*np.exp(12*x) + 45*np.exp(t*8*1j)*np.exp(8*x) + 45*np.exp(t*8*1j)*np.exp(10*x) + 18*np.exp(t*16*1j)*np.exp(4*x) + 9*np.exp(t*24*1j)*np.exp(2*x) + 18*np.exp(t*16*1j)*np.exp(14*x) + 64*np.exp(t*24*1j)*np.exp(6*x) + 36*np.exp(t*24*1j)*np.exp(8*x) + 36*np.exp(t*24*1j)*np.exp(10*x) + 64*np.exp(t*24*1j)*np.exp(12*x) + 18*np.exp(t*32*1j)*np.exp(4*x) + 9*np.exp(t*24*1j)*np.exp(16*x) + np.exp(t*24*1j)*np.exp(18*x) + 18*np.exp(t*32*1j)*np.exp(14*x) + 45*np.exp(t*40*1j)*np.exp(8*x) + 45*np.exp(t*40*1j)*np.exp(10*x) + 10*np.exp(t*48*1j)*np.exp(6*x) + 10*np.exp(t*48*1j)*np.exp(12*x)))  
    
    V = ut.real; W = ut.imag
    u = np.zeros((2*len(x)))
    u[::2] = V; u[1::2] = W  
    return u

def local_err_indicator(u1,u2,tol):
    m = len(u1); 
    vec = (u1-u2)/(tol*(1+np.max(np.array([np.abs(u1),np.abs(u2)]),0)))
    ep_n = np.sqrt(sum(vec**2)/m)
    return ep_n

#### ImEx methods

In [5]:
def ImEx_Sol(Mthdname,rkim, rkex, c, b, bhat, dt, f_stiff, f_non_stiff, T, u0, t0):    
    tt = np.zeros(1); t = t0; tt[0] = t # time   
    uu = np.zeros((1,np.size(u0))); uu[0,:] = u0.copy() # solution
    s = len(rkim); Rim = np.zeros((s,len(u0))); Rex = np.zeros((s,len(u0))); steps = 0

    while t < T and not np.isclose(t, T):
        clear_output(wait=True)
        if t + dt > T:
            dt = T - t
        for i in range(s):
            rhs = uu[-1].copy()
            if i>0:
                for j in range(i):
                    rhs += dt*(rkim[i,j]*Rim[j,:] + rkex[i,j]*Rex[j,:])

            Mat = I - dt*rkim[i,i]*M
            g_j = np.linalg.solve(Mat, rhs)
            Rim[i,:] = f_stiff(g_j,dx)
            Rex[i,:] = f_non_stiff(g_j,q)

        inc = dt*sum([ b[j]*(Rim[j]+Rex[j]) for j in range(s)]) 
        unew = uu[-1]+inc; t+= dt; tt = np.append(tt, t); steps += 1
        uu = np.append(uu, np.reshape(unew.copy(), (1,len(unew))), axis=0)  
        print("Method=FEM-%s: Step number = %d (time = %1.5f)"%(Mthdname,steps,tt[-1]))
        
    return tt, uu

In [6]:
def MR_ImEx_Sol(Mthdname, rkim, rkex, c, b, bhat, dt, f_stiff, f_non_stiff, T, u0, t0, inv):
    tt = np.zeros(1); t = t0; tt[0] = t;  # time 
    uu = np.zeros((1,np.size(u0))); uu[0,:] = u0.copy() # solution
    s = len(rkim); Rim = np.zeros((s,len(u0))); Rex = np.zeros((s,len(u0))) 
    gamma0 = np.zeros(inv); G = np.zeros((1,np.size(gamma0))); G[0,:] = gamma0.copy()
    steps = 0; no_ier_five = 0; no_ier_one = 0; no_ier_else = 0; tot_steps = 0; 
   
    while t < T and not np.isclose(t, T):
        clear_output(wait=True)
        tot_steps += 1
        if t + dt > T:
            dt = T - t
        for i in range(s):
            rhs = uu[-1].copy()
            if i>0:
                for j in range(i):
                    rhs += dt*(rkim[i,j]*Rim[j,:] + rkex[i,j]*Rex[j,:] )
            
            Mat = I - dt*rkim[i,i]*M
            g_j = np.linalg.solve(Mat, rhs)
            Rim[i,:] = f_stiff(g_j,dx)
            Rex[i,:] = f_non_stiff(g_j,q)
        
        inc1 = dt*sum([ b[i]*(Rim[i]+Rex[i]) for i in range(s)])  
        inc2 = dt*sum([ bhat[i]*(Rim[i]+Rex[i]) for i in range(s)])  
        unew = uu[-1]+inc1; 
        
        if inv == 1:
            inc = np.array([inc1]); E_old = np.array([eta1(uu[-1],dx)])
        elif inv == 2:
            inc = np.array([inc1,inc2]); E_old = np.array([eta1(uu[-1],dx), eta2(uu[-1],dx,q)])
           
        # fsolve
        ga_fsolve, info, ier, mesg = fsolve(rgam,gamma0,args=(unew,inc,E_old,inv,q),full_output=True,xtol = 1e-14)
        gamma = ga_fsolve; gamma0 = ga_fsolve

        steps += 1
        if ier == 1:
            no_ier_one += 1
        elif ier == 5:
            no_ier_five += 1
        else:
            no_ier_else += 1

        # relaxation solution
        unew = unew + np.dot(gamma,inc); t+=(1+sum(gamma))*dt 
        tt = np.append(tt, t); uu = np.append(uu, np.reshape(unew.copy(), (1,len(unew))), axis=0)  
        G = np.append(G, np.reshape(gamma.copy(), (1,len(gamma))), axis=0) 
               
        if inv == 1:
            print("FEM-%s: At step no = %d (time = %1.5f), ier = %d and γ1 = %1.6f \n"%(Mthdname,steps,tt[-1],ier,sum(gamma)))
        elif inv == 2:
            print("FEM-%s: At step no = %d (time = %1.5f), ier = %d and γ1+γ2  = %1.6f \n"%(Mthdname,steps,tt[-1],ier,sum(gamma)))
         
    return tot_steps, tt, uu, G, no_ier_one, no_ier_five, no_ier_else

#### ImEx methods with relaxation and varialbe time steps

In [7]:
def MR_ImEx_Var_Steps_Sol(Mthdname, rkim, rkex, c, b, bhat, dt, f_stiff, f_non_stiff, T, u0, t0, inv, alpha, em_or, tol):
    tt = np.zeros(1); t = t0; tt[0] = t;  # time 
    uu = np.zeros((1,np.size(u0))); uu[0,:] = u0.copy() # solution
    s = len(rkim); Rim = np.zeros((s,len(u0))); Rex = np.zeros((s,len(u0))) 
    gamma0 = np.zeros(inv); G = np.zeros((1,np.size(gamma0))); G[0,:] = gamma0.copy()
    steps = 0; no_ier_five = 0; no_ier_one = 0; no_ier_else = 0
    
    dt0 = dt; dts = np.zeros(1); dts[0] = dt; tot_steps = 0; 
    

    while t < T and not np.isclose(t, T):
        clear_output(wait=True)
        tot_steps += 1
        if t + dt > T:
            dt = T - t
        for i in range(s):
            rhs = uu[-1].copy()
            if i>0:
                for j in range(i):
                    rhs += dt*(rkim[i,j]*Rim[j,:] + rkex[i,j]*Rex[j,:] )
            
            Mat = I - dt*rkim[i,i]*M
            g_j = np.linalg.solve(Mat, rhs)
            Rim[i,:] = f_stiff(g_j,dx)
            Rex[i,:] = f_non_stiff(g_j,q)
        
        inc1 = dt*sum([ b[i]*(Rim[i]+Rex[i]) for i in range(s)])  
        inc2 = dt*sum([ bhat[i]*(Rim[i]+Rex[i]) for i in range(s)])  
        unew = uu[-1]+inc1; unew_em = uu[-1]+inc2;
        
        ep_n = local_err_indicator(unew,unew_em,tol)
        
        print('ep_n=%1.4f \n'%ep_n)
        if ep_n <= 1:
            if inv == 1:
                inc = np.array([inc1]); E_old = np.array([eta1(uu[-1],dx)])
            elif inv == 2:
                inc = np.array([inc1,inc2]); E_old = np.array([eta1(uu[-1],dx), eta2(uu[-1],dx,q)])

            # fsolve
            ga_fsolve, info, ier, mesg = fsolve(rgam,gamma0,args=(unew,inc,E_old,inv,q),full_output=True,xtol = 1e-14)
            gamma = ga_fsolve; gamma0 = ga_fsolve
            Err = np.linalg.norm(info['fvec'])
            if Err <= 1e-12:
                steps += 1
                if ier == 1:
                    no_ier_one += 1
                elif ier == 5:
                    no_ier_five += 1
                else:
                    no_ier_else += 1

                dts = np.append(dts,dt)
                # relaxation solution
                unew = unew + np.dot(gamma,inc); t+=(1+sum(gamma))*dt 
                tt = np.append(tt, t); uu = np.append(uu, np.reshape(unew.copy(), (1,len(unew))), axis=0)  
                G = np.append(G, np.reshape(gamma.copy(), (1,len(gamma))), axis=0) 
                dt = alpha*np.power((1/ep_n),1/(em_or+1))*dt # update time step for next time step
                print('Time step for the next iteration = %1.4f \n'%dt)
                if inv == 1:
                    print("FEM-%s: At step no = %d (time = %1.5f), ier = %d and γ1 = %1.6f \n"%(Mthdname,steps,tt[-1],ier,sum(gamma)))
                elif inv == 2:
                    print("FEM-%s: At step no = %d (time = %1.5f), ier = %d and γ1+γ2  = %1.6f \n"%(Mthdname,steps,tt[-1],ier,sum(gamma)))
            else:
                dt = dt/2
                print('fsolve failed. Ad-hoc proposed dt/2 = %1.4f \n'%dt)
        else: 
            dt = alpha*np.power((1/ep_n),1/(em_or+1))*dt # local error control fails, reject the step and recompute with controller predicted step
            print('Modified dt by step size control = %1.4f \n'%dt)

    return tot_steps, dts, tt, uu, G, no_ier_one, no_ier_five, no_ier_else

#### Time-splitting pseudospectral methods

In [8]:
# This is an implementation of an operator splitting 
# substeps
def sol_nl_part(u, q, dt):
    u_st = np.exp(1j*q*np.abs(u)**2*dt)*u
    return u_st
              
def sol_l_part(u, xi, dt):
    uhat = np.fft.fft(u)
    u_st_hat = np.exp(-1j*xi**2*dt)*uhat
    u_st = np.fft.ifft(u_st_hat)
    return u_st

def Op_Split_Exact_Solve(Mthdname,sol_nl_part,sol_l_part,q,xL,xR,N,t0,T,a,b,dt):
    # spatial discretization
    L = xR-xL; x = np.linspace(xL, xR, N+1)[:-1] # exclude the right boundary point
    dx = x[1]-x[0]; xi = np.fft.fftfreq(N) * N * 2*np.pi / L
    
    tt = np.zeros(1); t = t0; tt[0] = t # time
    u0 = np.zeros((2*N))
    for i, xc in enumerate(x):
        u0[i] = 1./np.cosh(xc)
        
    uu = np.zeros((1,np.size(u0))); uu[0,:] = u0.copy() # solution
    u_comp = u0[:N]+ (1j) *u0[N:]; steps = 0
    
    while t < T and not np.isclose(t, T):
        clear_output(wait=True)
        if t + dt > T:
            dt = T - t
        u_st = u_comp.copy()  
        for j in range(len(a)):
            u_st = sol_nl_part(u_st,q,a[j]*dt)
            u_st = sol_l_part(u_st,xi,b[j]*dt)
            
        u_comp = u_st.copy(); t+=dt
            
        unew = np.concatenate((u_comp.real,u_comp.imag)).astype('float64')
        tt = np.append(tt, t); uu = np.append(uu, np.reshape(unew.copy(), (1,len(unew))), axis=0)  
        steps += 1
        print("SP-%s: Step number = %d (time = %1.2f)"%(Mthdname,steps,tt[-1]))
    return tt, uu

In [9]:
# Initial profile
t0 = 0; u0 = np.zeros((2*m))
for i, xp in enumerate(x):
    u0[2*i] = 1./np.cosh(xp)

## Compute solutions by the numerical methods

In [10]:
# If a path doesn’t exist we create one
eqn = 'NLS_q%d_Sol%d_Inv%d'%(q,sol,inv)
import os
path1 = './Data/%s'%(eqn)
if not os.path.exists(path1):
   os.makedirs(path1)

#### Compute solution by ImEx with multiple relaxation and variable time steps

In [11]:
MR_EC_data = {'Mthds_MR_EC': Mthds_MR_EC,
        'Init: dt': DT,
        'Domain':'[%d,%d]'%(xL,xR),
        'm': m,
        't0': t0,
        'tf':'%1.1f'%(T),
        'q':'%d'%(q)}
MR_EC_df = pd.DataFrame(MR_EC_data)

f_stiff = rhs_stiff; f_non_stiff = rhs_non_stiff; 

# ImEx with relaxation and variable time steps
MR_EC_TS = []; MR_EC_dts = []; MR_EC_tt = []; MR_EC_uu = [];  
for idx in range(len(Mthds_MR_EC)):
    rkim, rkex, c, b, bhat = ImEx_schemes(Stage[idx],Order[idx],em_Or[idx],Sch_No[idx]);
    dt = DT[idx];  alpha = 0.9; tol = 1e-4; em_or = em_Or[idx]; 
    tot_steps, dts, tt, uu, G, IF_1,IF_5,IF_else = MR_ImEx_Var_Steps_Sol(Mthds_MR_EC[idx], rkim, rkex, c, b, bhat, dt, f_stiff, f_non_stiff, T, u0, t0, inv, alpha, em_or, tol) 
    MR_EC_df.at[idx,'R: ier = 1'] = int(IF_1); MR_EC_df.at[idx,'R: ier = 5'] = int(IF_5); MR_EC_df.at[idx,'R: ier = else'] = int(IF_else)
    MR_EC_TS.append(tot_steps); MR_EC_dts.append(dts); MR_EC_tt.append(tt); MR_EC_uu.append(uu)

ep_n=0.0207 

Time step for the next iteration = 0.0207 

FEM-ImEx4(MR)(EC): At step no = 436 (time = 5.00000), ier = 1 and γ1+γ2  = 0.000212 



In [12]:
# Computing reference solution corresponding to methods with relaxation and step size control
MR_EC_uu_ex = [];
for idx in range(len(Mthds_MR_EC)):
    MR_EC_t = MR_EC_tt[idx]; MR_EC_u_ex = []
    for i in range(len(MR_EC_t)):
        MR_EC_true_sol = NLS_True_Sol(MR_EC_t[i],x,q)
        MR_EC_u_ex.append(MR_EC_true_sol)
    MR_EC_uu_ex.append(MR_EC_u_ex)

In [13]:
# Saving Data
a_MR_EC_TS = np.empty(len(MR_EC_TS), dtype=object); a_MR_EC_TS[:] = MR_EC_TS
a_MR_EC_dts = np.empty(len(MR_EC_dts), dtype=object); a_MR_EC_dts[:] = MR_EC_dts

a_MR_EC_tt = np.empty(len(MR_EC_tt), dtype=object); a_MR_EC_tt[:] = MR_EC_tt
a_MR_EC_uu = np.empty(len(MR_EC_uu), dtype=object); a_MR_EC_uu[:] = MR_EC_uu
a_MR_EC_uu_ex = np.empty(len(MR_EC_uu_ex), dtype=object); a_MR_EC_uu_ex[:] = MR_EC_uu_ex

from pathlib import Path  
filepath = Path("./Data/%s/%s_m%d_T%1.1f_MR_EC_Mthd_Data.csv"%(eqn,eqn,m1,T),index = False)    
MR_EC_df.to_csv(filepath) 
# Numerical Solution by ImEx(MR)(EC) methods
np.save("./Data/%s/%s_m%d_T%1.1f_MR_EC_Tot_Steps.npy"%(eqn,eqn,m1,T), a_MR_EC_TS)
np.save("./Data/%s/%s_m%d_T%1.1f_MR_EC_Var_Time_Steps.npy"%(eqn,eqn,m1,T), a_MR_EC_dts)
np.save("./Data/%s/%s_m%d_T%1.1f_MR_EC_Time.npy"%(eqn,eqn,m1,T), a_MR_EC_tt)
np.save("./Data/%s/%s_m%d_T%1.1f_MR_EC_NumSol.npy"%(eqn,eqn,m1,T), a_MR_EC_uu)
# Reference Solution corresponding to ImEx(MR)(EC) methods
np.save("./Data/%s/%s_m%d_T%1.1f_MR_EC_TrueSol.npy"%(eqn,eqn,m1,T), a_MR_EC_uu_ex)

#### Compute solution by ImEx with multiple relaxation

In [14]:
MR_data = {'Mthds_MR': Mthds_MR,
        'MR: dt': DT,
        'Domain':'[%d,%d]'%(xL,xR),
        'm': m,
        't0': t0,
        'tf':'%1.1f'%(T),
        'q':'%d'%(q)}
MR_df = pd.DataFrame(MR_data)

f_stiff = rhs_stiff; f_non_stiff = rhs_non_stiff; 


# ImEx with multiple relaxation
MR_TS = []; MR_tt = []; MR_uu = [];  
for idx in range(len(Mthds_MR)):
    rkim, rkex, c, b, bhat = ImEx_schemes(Stage[idx],Order[idx],em_Or[idx],Sch_No[idx]);dt = DT[idx];  
    tot_steps, tt, uu, G, IF_1,IF_5,IF_else = MR_ImEx_Sol(Mthds_MR[idx], rkim, rkex, c, b, bhat, dt, f_stiff, f_non_stiff, T, u0, t0, inv) 
    MR_df.at[idx,'R: ier = 1'] = int(IF_1); MR_df.at[idx,'R: ier = 5'] = int(IF_5); MR_df.at[idx,'R: ier = else'] = int(IF_else)
    MR_TS.append(tot_steps); MR_tt.append(tt); MR_uu.append(uu)


FEM-ImEx4(MR): At step no = 499 (time = 5.00000), ier = 1 and γ1+γ2  = 0.000145 



In [15]:
# Saving Data
a_MR_TS = np.empty(len(MR_TS), dtype=object); a_MR_TS[:] = MR_TS
a_MR_tt = np.empty(len(MR_tt), dtype=object); a_MR_tt[:] = MR_tt
a_MR_uu = np.empty(len(MR_uu), dtype=object); a_MR_uu[:] = MR_uu

from pathlib import Path  
filepath = Path("./Data/%s/%s_m%d_T%1.1f_MR_Mthd_Data.csv"%(eqn,eqn,m1,T),index = False)    
MR_df.to_csv(filepath) 
# Numerical Solution by ImEx(MR)(EC) methods
np.save("./Data/%s/%s_m%d_T%1.1f_MR_Tot_Steps.npy"%(eqn,eqn,m1,T), a_MR_TS)
np.save("./Data/%s/%s_m%d_T%1.1f_MR_Time.npy"%(eqn,eqn,m1,T), a_MR_tt)
np.save("./Data/%s/%s_m%d_T%1.1f_MR_NumSol.npy"%(eqn,eqn,m1,T), a_MR_uu)

#### Compute solution by ImEx methods

In [16]:
B_data = {'Mthds_B': Mthds_B,
        'B: dt': DT,
        'Domain':'[%d,%d]'%(xL,xR),
        'm': m,
        't0': t0,
        'tf':'%1.1f'%(T),
        'q':'%d'%(q)}
B_df = pd.DataFrame(B_data)

f_stiff = rhs_stiff; f_non_stiff = rhs_non_stiff; 

# ImEx with multiple relaxation
B_tt = []; B_uu = [];  
for idx in range(len(Mthds_B)):
    rkim, rkex, c, b, bhat = ImEx_schemes(Stage[idx],Order[idx],em_Or[idx],Sch_No[idx]);dt = DT[idx];  
    tt, uu = ImEx_Sol(Mthds_B[idx],rkim, rkex, c, b, bhat, dt, f_stiff, f_non_stiff, T, u0, t0)
    B_tt.append(tt); B_uu.append(uu)

Method=FEM-ImEx4: Step number = 500 (time = 5.00000)


In [17]:
# Computing reference solution corresponding to ImEx methods
B_uu_ex = [];
for idx in range(len(Mthds_B)):
    B_t = B_tt[idx]; B_u_ex = []
    for i in range(len(B_t)):
        B_true_sol = NLS_True_Sol(B_t[i],x,q)
        B_u_ex.append(B_true_sol)
    B_uu_ex.append(B_u_ex)

In [18]:
# Saving Data
a_B_tt = np.empty(len(B_tt), dtype=object); a_B_tt[:] = B_tt
a_B_uu = np.empty(len(B_uu), dtype=object); a_B_uu[:] = B_uu
a_B_uu_ex = np.empty(len(B_uu_ex), dtype=object); a_B_uu_ex[:] = B_uu_ex

from pathlib import Path  
filepath = Path("./Data/%s/%s_m%d_T%1.1f_B_Mthd_Data.csv"%(eqn,eqn,m1,T),index = False)    
B_df.to_csv(filepath) 
# Numerical Solution by ImEx methods
np.save("./Data/%s/%s_m%d_T%1.1f_B_Time.npy"%(eqn,eqn,m1,T), a_B_tt)
np.save("./Data/%s/%s_m%d_T%1.1f_B_NumSol.npy"%(eqn,eqn,m1,T), a_B_uu)
# Reference Solution corresponding to ImEx(MR)(EC) methods
np.save("./Data/%s/%s_m%d_T%1.1f_B_TrueSol.npy"%(eqn,eqn,m1,T), a_B_uu_ex)

#### Compute solution by time splitting pseudospectral methods

In [19]:
SP_data = {'Mthds_SP': Mthds_SP,
        'SP: dt': SP_DT,
        'Domain':'[%d,%d]'%(xL,xR),
        'm': m,
        't0': t0,
        'tf':'%1.1f'%(T),
        'q':'%d'%(q)}
SP_df = pd.DataFrame(SP_data)

f_stiff = rhs_stiff; f_non_stiff = rhs_non_stiff; 

# Time splitting methods
SP_tt = []; SP_uu = [];  
for idx in range(len(Mthds_SP)):
    OpSp_a, OpSp_b = Op_Sp_Coeff(SP_Stage[idx],SP_Order[idx],SP_Sch_No[idx]); dt = SP_DT[idx];  
    tt, uu = Op_Split_Exact_Solve(Mthds_SP[idx],sol_nl_part,sol_l_part,q,xL,xR,N,t0,T,OpSp_a,OpSp_b,dt)
    SP_tt.append(tt); SP_uu.append(uu)

SP-AK4: Step number = 500 (time = 5.00)


In [20]:
# Saving Data
a_SP_tt = np.empty(len(SP_tt), dtype=object); a_SP_tt[:] = SP_tt
a_SP_uu = np.empty(len(SP_uu), dtype=object); a_SP_uu[:] = SP_uu

from pathlib import Path  
filepath = Path("./Data/%s/%s_m%d_T%1.1f_SP_Mthd_Data.csv"%(eqn,eqn,m1,T),index = False)    
SP_df.to_csv(filepath) 
# Numerical Solution by ImEx(MR)(EC) methods
np.save("./Data/%s/%s_m%d_T%1.1f_SP_Time.npy"%(eqn,eqn,m1,T), a_SP_tt)
np.save("./Data/%s/%s_m%d_T%1.1f_SP_NumSol.npy"%(eqn,eqn,m1,T), a_SP_uu)


## End